In [ ]:
import matplotlib.pyplot as plt
import gc
import time
plt.clf()
plt.close()
#plt.gcf().clear()  # 現在のFigureをクリア  # Figureオブジェクトを削除
gc.collect()  # ガベージコレクションを強制実行
#time.sleep(60)

In [ ]:
#https://qiita.com/yuitomo/items/dc600abdd04931646e1e

import numpy as np
#import keras
import tensorflow
import glob
import os
#from tensorflow.keras.utils import np_utils
from PIL import Image
from sklearn.model_selection import StratifiedKFold



#from tensorflow.keras.models import Sequential
#from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, KFold

# -------------------------------------------------------------------------------------
#                        初期設定部
# -------------------------------------------------------------------------------------

# GrayScaleのときに1、COLORのときに3にする
COLOR_CHANNEL = 3

# 入力画像サイズ(画像サイズは正方形とする)
INPUT_IMAGE_SIZE = 224

# 訓練時のバッチサイズとエポック数
BATCH_SIZE = 32
EPOCH_NUM = 300

db = -16

# 使用する訓練画像の入ったフォルダ(ルート)
TRAIN_PATH =  "C:/Users/Casper1/2023/okumura/bubblejet/dataset_noized_fourier_resize/"+str(db)+"db/0.5MPa"
# 使用する訓練画像の各クラスのフォルダ名
folder = ["100", "200", "250"]

save_directory = "C:/Users/Casper1/2023/okumura/bubblejet/dataset_noized_fourier_resize/result/0.5MPa"


# CLASS数を取得する
CLASS_NUM = len(folder)
print("クラス数 : " + str(CLASS_NUM))

#testを除いた中からvalidationの割合
validation_rate = 0.2

# 分割数を設定
k = 5

#分割交差検証の平均算出用
test_loss = [0]*k
test_accuracy = [0]*k
sum_accuracy = 0
sum_val_accuracy = 0
sum_test_accuracy = 0
sum_loss = 0
sum_val_loss = 0
sum_test_loss = 0

# -------------------------------------------------------------------------------------
#                        訓練画像入力部
# -------------------------------------------------------------------------------------

# 各フォルダの画像を読み込む
v_image = []
v_label = []
for index, name in enumerate(folder):
    dir = TRAIN_PATH + "\\" + name
    files = glob.glob(dir + "\\*.png")
    print(dir)
    for i, file in enumerate(files):
        if COLOR_CHANNEL == 1:
            img = load_img(file, color_mode = "grayscale", target_size=(INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE))
        elif COLOR_CHANNEL == 3:
            img = load_img(file, color_mode = "rgb", target_size=(INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE))
        array = img_to_array(img)
        v_image.append(array)
        v_label.append(index)

v_image = np.array(v_image)
v_label = np.array(v_label)


# imageの画素値をint型からfloat型にする
v_image = v_image.astype('float32')
# 画素値を[0～255]⇒[0～1]とする
v_image = v_image / 255.0

# データの前処理
# ラベルを0から3の整数に変換
label_binarizer = LabelBinarizer()
#v_label = label_binarizer.fit_transform(v_label)

#StratifiedKFoldの設定
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

from keras import backend as K
from keras.layers import Conv2D, Add, Input, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense
from keras.models import Model

def shortcut(conv, residual):
    conv_shape = K.int_shape(conv)
    residual_shape = K.int_shape(residual)
    if conv_shape != residual_shape:
        residual = Conv2D(filters=conv_shape[3], kernel_size=(1, 1), strides=(2, 2))(residual)
    return Add()([conv, residual])

def res_blocks(x,filter,stride=1):
    conv = Conv2D(filters=filter, kernel_size=(3, 3), strides=(stride, stride), padding="same", kernel_initializer='he_normal')(x)
    conv = BatchNormalization()(conv)
    conv = Activation("relu")(conv)
    conv = Conv2D(filters=filter, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal')(conv)
    conv = BatchNormalization()(conv)
    short_cut = shortcut(conv, x)
    conv = Activation("relu")(short_cut)
    return conv


class ResNet18:
    def __init__(self, input_shape, nb_classes):
        self.input_shape = input_shape
        self.nb_classes = nb_classes
        self.model = self.make_model()

    def make_model(self):
        inputs = Input(self.input_shape)
        x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding="same", kernel_initializer='he_normal')(inputs)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(x)

        x = res_blocks(x, 64)
        x = res_blocks(x, 64)

        x = res_blocks(x, 128, stride=2)
        x = res_blocks(x, 128)

        x = res_blocks(x, 256, stride=2)
        x = res_blocks(x, 256)

        x = res_blocks(x, 512, stride=2)
        x = res_blocks(x, 512)
        
        x = GlobalAveragePooling2D()(x)
        outputs = Dense(units=self.nb_classes, activation='softmax')(x)
        ResNetModel = Model(inputs=inputs, outputs=outputs)
        return ResNetModel

def build_model(input_shape=(224, 224, 3), nb_classes=3):
    return ResNet18(input_shape=input_shape, nb_classes=nb_classes).model


for fold_num, (train_index, test_index) in enumerate(skf.split(v_image, v_label)):
    print(f"Fold {fold_num}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    file_path1 = os.path.join(save_directory,f"{db}db_{fold_num}fold_result")
    file_path2 = os.path.join(save_directory,f"{db}db_{fold_num}fold_confmat")
    # 訓練データとラベル
    train_data, train_labels = v_image[train_index], v_label[train_index]
    
    # テストデータとラベル
    test_data, test_labels = v_image[test_index], v_label[test_index]
    
    #labelbinarizer
    train_labels = label_binarizer.fit_transform(train_labels)
    test_labels = label_binarizer.fit_transform(test_labels)
    
    #シャッフル
    train_data, train_labels = shuffle(train_data, train_labels, random_state=0)
    
    model = ResNet18(input_shape=(224,224,3), nb_classes=3).model
    # モデルのコンパイル
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    # 訓練
    history = model.fit(train_data, train_labels, validation_split = validation_rate, batch_size=BATCH_SIZE, epochs=EPOCH_NUM)
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    Accuracy = history.history['accuracy']
    val_Accuracy = history.history['val_accuracy']
    epochs = range(1, len(loss) + 1)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.yscale('log')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(epochs, Accuracy, 'b', label='Training Accuracy')
    plt.plot(epochs, val_Accuracy, 'r', label='Validation Accuracy')
    plt.yscale('log')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(file_path1,dpi=180.7, bbox_inches='tight',pad_inches = 0)
    plt.show()
    
    #print(Accuracy[-1])
    

    # 訓練データの予測
    test_predictions = model.predict(test_data)
    # 予測結果からクラスを選択
    test_predicted_classes = np.argmax(test_predictions, axis=1)
    # 正解ラベルをクラスに戻す
    test_true_classes = np.argmax(test_labels, axis=1)
    # 混同行列を計算
    confusion = confusion_matrix(test_true_classes, test_predicted_classes)
    # 混同行列を可視化
    display = ConfusionMatrixDisplay(confusion, display_labels=folder)
    display.plot(cmap=plt.cm.Blues, values_format="d")
    plt.title("Confusion Matrix (Training Data)")
    plt.savefig(file_path2,dpi=180.7, bbox_inches='tight',pad_inches = 0)
    plt.show()

    
    score = model.evaluate(test_data, test_labels, verbose=0)
    print(len(test_data))
    test_loss[fold_num]=score[0]
    test_accuracy[fold_num]=score[1]
    print('Loss:', score[0])
    print('Accuracy:', score[1])
    
    sum_accuracy = sum_accuracy + Accuracy[-1]
    sum_val_accuracy = sum_val_accuracy + val_Accuracy[-1]
    sum_test_accuracy = sum_test_accuracy + score[1]
    sum_loss = sum_loss + loss[-1]
    sum_val_loss = sum_val_loss + val_loss[-1]
    sum_test_loss = sum_test_loss + score[0]
        
ave_accuracy = sum_accuracy/k
ave_val_accuracy = sum_val_accuracy/k
ave_test_accuracy = sum_test_accuracy/k
ave_loss = sum_loss/k
ave_val_loss = sum_val_loss/k
ave_test_loss = sum_test_loss/k

for j in range(k):
    print(f"hold{j+1}_test_loss:{test_loss[j]}")

    
for j in range(k):
    print(f"hold{j+1}_test_accuracy:{test_accuracy[j]}")
    

print(f"平均accuracy: {ave_accuracy}")
print(f"平均val_accuracy: {ave_val_accuracy}")
print(f"平均test_accuracy: {ave_test_accuracy}")
print(f"平均loss:{ave_loss}")
print(f"平均val_loss:{ave_val_loss}")
print(f"平均test_loss:{ave_test_loss}")

text_name = "C:/Users/Casper1/2023/okumura/bubblejet/dataset_noized_fourier_resize/result/0.5MPa/"+str(db)+"db_result.txt"

with open(text_name,'w')as file:
    for j in range(k):
        file.write(f"hold{j+1}_test_loss:{test_loss[j]}\n")
    for l  in range(k):
        file.write(f"hold{j+1}_test_loss:{test_accuracy[j]}\n")
    file.write(f"平均accuracy: {ave_accuracy}\n")
    file.write(f"平均val_accuracy: {ave_val_accuracy}\n")
    file.write(f"平均test_accuracy: {ave_test_accuracy}\n")
    file.write(f"平均loss:{ave_loss}\n")
    file.write(f"平均val_loss:{ave_val_loss}\n")
    file.write(f"平均test_loss:{ave_test_loss}\n")